In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set('spark.sql.autoBroadcastJoinThreshold', -1)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/07 15:25:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!hdfs dfs -mkdir /tmp/crime

2025-01-07 15:25:22,545 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!hdfs dfs -put crime.csv /tmp/crime

2025-01-07 15:25:26,165 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!hdfs dfs -put offense_codes.csv /tmp/crime

2025-01-07 15:25:32,443 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
crime = spark \
    .read \
    .option('header', 'true') \
    .csv('/tmp/crime/crime.csv')

crime.show(5)

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|     STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|       00619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|       01402|           Vandalism|           VANDALISM|     C11|           347|    null|201

In [8]:
offense_codes = spark \
    .read \
    .option('header', 'true') \
    .csv('/tmp/crime/offense_codes.csv') \
    .withColumn('code', F.lpad('code', 5, '0'))

offense_codes.show(5)

+-----+--------------------+
| code|                NAME|
+-----+--------------------+
|00612|LARCENY PURSE SNA...|
|00613| LARCENY SHOPLIFTING|
|00615|LARCENY THEFT OF ...|
|01731|              INCEST|
|03111|LICENSE PREMISE V...|
+-----+--------------------+
only showing top 5 rows



In [9]:
stage0 = crime \
    .join(F.broadcast(offense_codes), \
          crime['offense_code'] == offense_codes['code'], \
          how='left') \
    .selectExpr('incident_number', 'offense_code', 'offense_code_group',
                'name as offense_name', 'offense_description', 'district',
                'reporting_area', 'shooting', 'occurred_on_date',
                'year', 'month', 'day_of_week',
                'hour', 'ucr_part', 'street',
                'lat', 'long', 'location')

stage0.show(5)

+---------------+------------+------------------+--------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|incident_number|offense_code|offense_code_group|        offense_name|offense_description|district|reporting_area|shooting|   occurred_on_date|year|month|day_of_week|hour|  ucr_part|     street|        lat|        long|            location|
+---------------+------------+------------------+--------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|       00619|           Larceny|LARCENY OTHER $20...| LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070945|       00619|      

In [14]:
stage1 = stage0 \
    .where(F.col('district').isNotNull()) \
    .withColumn('year_month', F.concat_ws('-', F.col('year'), F.col('month'))) \
    .groupBy('district', 'year_month') \
    .agg(F.count('incident_number').alias('crimes_subtotal')) \
    .select('district', 'year_month', 'crimes_subtotal')

stage1.show(5)

+--------+----------+---------------+
|district|year_month|crimes_subtotal|
+--------+----------+---------------+
|      D4|    2016-9|           2116|
|      D4|   2015-10|           1958|
|      A7|    2016-8|            661|
|     C11|    2016-6|           2003|
|      B2|    2015-6|           1264|
+--------+----------+---------------+
only showing top 5 rows



In [15]:
stage2 = stage1 \
    .groupBy('district') \
    .agg(F.sum('crimes_subtotal').alias('crimes_total'), \
         F.median('crimes_subtotal').alias('crimes_monthly')) \
    .select('district', 'crimes_total', 'crimes_monthly')

stage2.show(5)

+--------+------------+--------------+
|district|crimes_total|crimes_monthly|
+--------+------------+--------------+
|      C6|       42353|        1074.5|
|      B2|       89502|        2338.5|
|     C11|       76812|        2002.5|
|     E13|       31581|         799.5|
|      B3|       64731|        1666.5|
+--------+------------+--------------+
only showing top 5 rows



In [18]:
stage3 = stage0 \
    .where(F.col('district').isNotNull()) \
    .groupBy('district') \
    .agg(F.avg('lat').alias('avg_lat'), \
         F.avg('long').alias('avg_long')) \
    .selectExpr('district', 'avg_lat as lat', 'avg_long as lng')

stage3.show(5)

+--------+------------------+------------------+
|district|               lat|               lng|
+--------+------------------+------------------+
|      C6|  42.2127825349144|-70.85656945781402|
|      B2| 42.31593668114477|-71.07556314050598|
|     C11| 42.29307710447023|-71.05195457998221|
|     E13| 42.31083916422159|-71.09944147664042|
|      B3|42.282705915481436|-71.07838836128927|
+--------+------------------+------------------+
only showing top 5 rows



In [19]:
stage4 = stage2 \
    .join(stage3, \
          on='district', \
          how='left') \
    .orderBy(F.asc('district')) \
    .select('district', 'crimes_total', 'crimes_monthly', 'lat', 'lng')

stage4.show(12)

+--------+------------+--------------+------------------+------------------+
|district|crimes_total|crimes_monthly|               lat|               lng|
+--------+------------+--------------+------------------+------------------+
|      A1|       66275|        1673.5| 42.32935895752824| -71.0169132660099|
|     A15|       11588|         288.5|42.184472217986624|-70.75327686373568|
|      A7|       24488|         627.0| 42.35890784566083|-71.00143435045631|
|      B2|       89502|        2338.5| 42.31593668114477|-71.07556314050598|
|      B3|       64731|        1666.5|42.282705915481436|-71.07838836128927|
|     C11|       76812|        2002.5| 42.29307710447023|-71.05195457998221|
|      C6|       42353|        1074.5|  42.2127825349144|-70.85656945781402|
|     D14|       36030|         916.5|42.342757123436165|-71.12993321509198|
|      D4|       77104|        2001.0| 42.34113400538648|-71.07698375791774|
|     E13|       31581|         799.5| 42.31083916422159|-71.09944147664042|

In [20]:
spark.stop()